In [1]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib
import lightgbm as lgb
import numpy as np
print("everything import well")

everything import well


In [2]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')


In [4]:
import re
import nltk
from nltk.corpus import stopwords

# Ensure stopwords are available
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# English stopwords
english_stopwords = set(stopwords.words('english'))

#Somali stopwords (custom – can be expanded carefully)
somali_stopwords = {
    "iyo", "waa", "in", "ka", "ku", "si", "ayaa", "ma", "haa", "leh", "loo",
    "la", "u", "wax", "badan", "ahay", "karo", "mid", "kuma", "wuu", "waxa"
}

# Combine stopwords
all_stopwords = english_stopwords.union(somali_stopwords)

def clean_text(text):
    # 1. Convert to lowercase
    text = text.lower()
    
    # 2. Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 3. Tokenize (simple split)
    words = text.split()
    
    # 4. Remove English + Somali stopwords
    words = [word for word in words if word not in all_stopwords]
    
    # 5. Join words back to text
    return " ".join(words)




In [6]:
df = pd.read_csv("extended_spam_dataset.csv")
df.columns = ["label", "message"]                       # ensure correct format
df = df[df["label"].isin(["ham", "spam"])]             # keep only valid labels


In [7]:
df['clean_message'] = df['message'].apply(clean_text)
df[['message', 'clean_message']].head()


,message,clean_message
0,22 days to kick off! For Euro2004 U will be ke...,days kick euro kept date latest news results d...
1,Dai what this da.. Can i send my resume to thi...,dai da send resume id
2,Al he does is moan at me if n e thin goes wron...,al moan n e thin goes wrong faultal de argumen...
3,Maanta waan mashquulsanahay.,maanta waan mashquulsanahay
4,I'm in class. Will holla later,im class holla later


In [8]:
df['label'] = df['label'].map({'ham': 0, 'spam': 1})
df['label'].value_counts()


label
0    5325
1    1247
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X = df['clean_message']   # use consistent column name
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [10]:
print("Train distribution:")
print(y_train.value_counts())

print("\nTest distribution:")
print(y_test.value_counts())


Train distribution:
label
0    4260
1     997
Name: count, dtype: int64

Test distribution:
label
0    1065
1     250
Name: count, dtype: int64


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_features=6000
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(X_train_vec.shape)
print(X_test_vec.shape)


(5257, 6000)
(1315, 6000)


In [12]:
# ==============================
# 6A. TRAIN SVM MODEL
# ==============================
svm_model = SVC(
    kernel="linear",
    probability=True,
    class_weight="balanced",
    C=2
)


svm_model.fit(X_train_vec, y_train)


svm_pred = svm_model.predict(X_test_vec)

print("\n=== SVM Accuracy ===")
print(accuracy_score(y_test, svm_pred))

print("\n=== SVM Report ===")
print(classification_report(y_test, svm_pred))



=== SVM Accuracy ===
0.9863117870722433

=== SVM Report ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1065
           1       0.97      0.96      0.96       250

    accuracy                           0.99      1315
   macro avg       0.98      0.98      0.98      1315
weighted avg       0.99      0.99      0.99      1315



In [13]:
# ==============================
# 6B. TRAIN LIGHTGBM MODEL (CORRECT)
# ==============================
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# LightGBM datasets (labels already numeric)
lgb_train = lgb.Dataset(X_train_vec, label=y_train)
lgb_eval = lgb.Dataset(X_test_vec, label=y_test, reference=lgb_train)

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting_type": "gbdt",
    "num_leaves": 40,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "verbose": -1
}

lightgbm_model = lgb.train(
    params,
    lgb_train,
    num_boost_round=300,
    valid_sets=[lgb_eval],
    callbacks=[lgb.early_stopping(stopping_rounds=30)]
)

# Predictions
lgb_pred_prob = lightgbm_model.predict(X_test_vec)
lgb_pred = (lgb_pred_prob >= 0.5).astype(int)

print("\n=== LightGBM Accuracy ===")
print(accuracy_score(y_test, lgb_pred))

print("\n=== LightGBM Report ===")
print(classification_report(y_test, lgb_pred))


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[211]	valid_0's binary_logloss: 0.0773083

=== LightGBM Accuracy ===
0.9787072243346008

=== LightGBM Report ===
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1065
           1       0.95      0.94      0.94       250

    accuracy                           0.98      1315
   macro avg       0.97      0.96      0.97      1315
weighted avg       0.98      0.98      0.98      1315



In [14]:
# ==============================
# 7. SAVE MODELS
# ==============================
joblib.dump(svm_model, "svm_model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")
lightgbm_model.save_model("lightgbm_model.txt")

print("\nModels saved successfully!")


Models saved successfully!


In [19]:
# ==============================
# 1. IMPORT LIBRARIES
# ==============================
import re
import joblib
import nltk
from nltk.corpus import stopwords

# ==============================
# 2. ENSURE STOPWORDS
# ==============================
try:
    nltk.data.find("corpora/stopwords")
except LookupError:
    nltk.download("stopwords")

# ==============================
# 3. STOPWORDS (ENGLISH + SOMALI)
# ==============================
english_stopwords = set(stopwords.words("english"))

somali_stopwords = {
    "iyo", "waa", "in", "ka", "ku", "si", "ayaa", "ma", "haa", "leh", "loo",
    "la", "u", "wax", "badan", "ahay", "karo", "mid", "kuma", "wuu", "waxa"
}

all_stopwords = english_stopwords.union(somali_stopwords)

# ==============================
# 4. CLEAN TEXT FUNCTION
# ==============================
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in all_stopwords]
    return " ".join(words)

# ==============================
# 5. LOAD SAVED MODEL & VECTORIZER
# ==============================
svm_model = joblib.load("svm_model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

print("Model and vectorizer loaded successfully ✅")

# ==============================
# 6. TEST MESSAGE
# ==============================
test_message = "salaman asxbta ina fcn tihin rajena beri ina dhibcaha kaso qadano fuad aan isku dayno oo shaqadi malin dhawed la rabay aan geyno"

# Clean text
cleaned_message = clean_text(test_message)

# Vectorize
message_vec = vectorizer.transform([cleaned_message])

# Predict
prediction = svm_model.predict(message_vec)[0]
probabilities = svm_model.predict_proba(message_vec)[0]

spam_prob = probabilities[1]
ham_prob = probabilities[0]

# ==============================
# 7. OUTPUT RESULT (CLEAR & CORRECT)
# ==============================
print("\nOriginal Message:")
print(test_message)

print("\nCleaned Message:")
print(cleaned_message)

print("\n🚨Confidence score:")
if prediction == 1:
    print(f"🚨 SPAM ({spam_prob:.2f})")
else:
    print(f"✅ HAM ({ham_prob:.2f})")


Model and vectorizer loaded successfully ✅

Original Message:
salaman asxbta ina fcn tihin rajena beri ina dhibcaha kaso qadano fuad aan isku dayno oo shaqadi malin dhawed la rabay aan geyno

Cleaned Message:
salaman asxbta ina fcn tihin rajena beri ina dhibcaha kaso qadano fuad aan isku dayno oo shaqadi malin dhawed rabay aan geyno

🚨Confidence score:
✅ HAM (0.98)
